# 時系列分析

## 本メモ（時系列分析と状態空間モデルの基礎　RとStanで学ぶ理論と実装）

## 必要知識
- データ生成過程（DGP）
    - 今回取得できたデータは、パラレルワールドの中の1回で取得できたデータだと考える
    - →　母集団は、パラレルワールドの全体
    - データ生成過程とは、取得できたデータがどのような確率分布から取得できたのかを考えること
- 時系列分析の目的：時系列モデルを推定すること
- ホワイトノイズ
    - 未来予測の情報がない雑音
    - 期待値が０で、分散が一定、自己相関が０
- ランダムウォーク
    - iid系列の累積和
- 構造
    - 特徴はデータの前後に関係があること
    - 自己相関をグラフにする　→　コレログラム
    - 時系列データ　＝　短期の自己相関
           ＋周期変動
           ＋トレンド
           ＋外因性
           ＋ホワイトノイズ
- 相関
    - 自己相関：k時点前との間にある相関
    - 偏自己相関：k-1時点との相関をなくしたk時点との相関
                - 「1時点まででは表現できなかった残り」同士で相関をとる
    - iid系列
        - データが独立である条件
        - ホワイトノイズとの違い
            - iidは平均０とは限らない
            - ホワイトノイズは独立のみ。iidは独立かつ同一分布
            - iidは強定常、ホワイトノイズは弱定常
            - 錯乱項を発生させるには、 ホワイトノイズで十分
            - ランダムウォークのみiidを仮定（ARやMAはホワイトノイズを仮定）
        - iid系列　　　　　→　強定常：常に同じ確率分布
        - ホワイトノイズ　→　弱定常：時間差のみに依存
    - 定常性
        - 平均と自己共分散が一定であること
    - そもそも時系列は分析しずらいので、「分析しやすいデータ」のみを扱う　→　定常性
        - 定常過程　→　平均が一定　
            - 基本的なデータはトレンドがあったり、季節性があったりと、ほぼ定常ではない
        - でも定常仮定でないと分析できないので、差分をとったりして、差分系列を作る
            - 原系列　→　非定常過程　かつ　差分系列　→　定常過程　の場合を単位根過程（1次和分過程）
        - ランダムウォークはホワイトノイズの累積和なので非定常だが、差分を取れば定常になる
            
****
            
##  分析手法
 - Box-Jenkins法の分析手順<br>
    Step1　：　データを分析やすいように変換<br>
    Step2　：　データをモデル化<br>
    Step3　：　モデルの評価<br>
    Step4　：　モデルで予測する<br>

    - ARモデル
        - 前地点のデータと相関がある場合
        - yt = c + φ1yt-1 + εt　c：定数　φ：係数
        - φ > 1：過去データの影響がずっと残る
        - φ = 1 ：ランダムウォーク
        - φ  < 1：過去のデータの影響がほぼなくなる
        -　常に定常ではない（|φ|<1のみ定常）
    - MAモデル
        - 自己相関がある場合
        - 常に定常
        - 反転可能性によって、「良い性質」かを判断可能
            - 反転可能性とは、「予測誤差（ε)の大小を、過去のデータから判断できるかどうか」
            - 反転可能性の条件　→ 特性方程式において、解の絶対値が1より大きいこと
            - 上記を満たすと、予測誤差（ε)の大小を、過去のデータから判断できる
            - ※εを過去データのΣで表現する事ができる為
    - ARとMAの違い
        - AR：過去のある時点における自分のデータ、偏自己相関に対応
        - MA：過去の複数地点の移動平均における自分のデータ、自己相関に対応
    - ARIMA
        - 和分過程の場合、ARMAは使えないので、ARIMAにして差分系列にしてから使う
    - SARIMA
        - ARIMAに季節性を取り入れたモデル
        - SARIMAでは、ARIMAで使った「1時点前との変化」＋「1季節前との変化」によってモデル化
    - ARIMAX
        - 「回帰」要素を取り入れたARIMA
        - ダミー変数を使って、曜日や祝日の効果を組み込める
- 分析モデルの次数の決め方
    - ３つの作業
        - 次数の決定：モデル選択
        - 差分の決定：単位根検定
        - モデルの評価
    - 次数の決定：AICを使って、手当たり次第に評価してみる
        - AICを使う理由：過学習を抑えるため
    - 差分の決定：KPSS検定かADF検定
        - KPSS
            - 帰無仮説：単位根なし
            - 対立仮設：単位根あり
        - ADF
            - 帰無仮説：単位根あり
            - 対立仮設：単位根なし
        - 使い分け
            - ADF：検定回帰式に、「被説明変数のラグ項」を含める
    - 定常性と反転可能性の確認（ARMAの場合）
        - 定常性
            - MA項は常に定常なため、AR項の定常性かどうかがARMAの定常性となる
            - AR項は定常なら常に反転可能なため、MA項の反転可能かどうかがARMAの反転可能な条件となる
    - ARIMAの次数決定アルゴリズム<br>
        1 : KPSS検定にて、単位根の有無を調べる<br>
        2 : 差分系列に対して、ARIMAモデルのp,q次数を手当たり次第作成<br>
        3 : 定常性・反転可能性をチェック<br>
        ４ : 定常性・反転可能性チェックを通った中でAICが最も低いモデルを選ぶ<br>
    - モデル評価
        - 残差がホワイトノイズである　→　うまくモデル化できている
        - 残差がホワイトノイズでない　→　まだモデルに反映できる要素が残っている
        - 残差に自己相関があるかを検定（ホワイトノイズかどうか）
            - リュング・ボックス検定
                - 帰無仮説：k時点までのデータが全て自己相関0
                - 対立仮設：いずれかが０ではない
        - 当てはめ残差が正規分布かを検定（残差は正規分布を仮定したホワイトノイズのため）
            - ジャック・ベラ検定
                - 帰無仮説：正規分布
                - 対立仮設：正規分布でない
***

## 時系列分析で気をつける点
- めも：最小二乗法では時系列データがうまくいかない理由
    - データが独立ではない場合が多い　→　昨日の売上が今日の売上に関係している場合が多い
- みせかけの回帰
    - 原因は、残差に自己相関があること
        - 最小二乗法が有効でなくなる
    - 最小二乗法（OLS）ではなく、一般最小二乗法（GLS）を使う → Prais–Winsten法
        - AR(1)とすると、yt = β1 + β2xt + εt
        - 残差εから自己相関を想定した回帰式を作り、その係数からytの推定値を作る
        - ytの推定値は残差の自己相関を取り除いているので、時系列モデルを組む
    - ダービン・ワトソン検定
        - 線形回帰の結果に対して、自己相関の有無を調べる
        - 帰無仮説：自己相関０
        - 対立仮設：０ではない
    - 防ぐ方法
        - 過去のデータをモデルに組み込み、データの持つ自己相関を表現するモデルを作成（ARIMAXやVAR）
        - 差分系列の回帰分析をとる
            - 共和分
                - 以下を満たす
                    - 2つのデータどちらも単位根過程
                    - 2つのデータは線形結合すると単位根過程でなくなる
- VAR
    - 多変量の時系列モデル
        - お互いに影響を及ぼす時に考える　※どちらか一方的な場合はARIMAXを使う
        - 例：収入と消費　→　収入が増えれば消費が増える事が仮定できる
        - yt = c + φ1yt-1 + ... + φpyt-p + εt （t:n次ベクトル、c:n×1次定数ベクトル、φ:n×n次元係数行列）
    - グレンジャー因果検定
        - 相手のデータを使う事で、予測精度が上がるかどうか　→ 直接の因果ではなく、ほぼ相関に近いよう
        - 今年の収入　＝　φ11去年の収入　＋　φ12去年の消費　＋　c + ε<br>
          今年の収入　＝　φ21去年の収入　　　　　　　　　　　＋ c +　ε
           

***

## 不明点
### 21/12/29時点
 - 状態空間モデルとは？
 - カルマンフィルタとは？
 - 錯乱項ならホワイトノイズでも良いのに、ランダムウォークのみiidを仮定する意味は？
 - 差分を決める際に、KPSSとADFの両検定における違いは？
 - 時系列モデルで、実装している例ってある？

***

## 関係性
### 21/12/29時点
 - ARとVRの違い<br>
     - AR：過去の自分と関係性がある
     - MA：自己相関がある